In [ ]:
def parse_tripinfo(tripinfo_path):
    """
    Parse tripinfo XML file and extract KPIs.
    Returns: dictionary with aggregated metrics
    """
    if not os.path.exists(tripinfo_path):
        return None
    
    tree = ET.parse(tripinfo_path)
    root = tree.getroot()
    
    trips = []
    for tripinfo in root.findall('tripinfo'):
        trip = {
            'id': tripinfo.get('id'),
            'duration': float(tripinfo.get('duration')),
            'waitingTime': float(tripinfo.get('waitingTime')),
            'timeLoss': float(tripinfo.get('timeLoss')),
            'routeLength': float(tripinfo.get('routeLength')),
            'departSpeed': float(tripinfo.get('departSpeed')),
            'arrivalSpeed': float(tripinfo.get('arrivalSpeed')),
        }
        trips.append(trip)
    
    if not trips:
        return None
    
    df = pd.DataFrame(trips)
    
    metrics = {
        'num_vehicles': len(df),
        'avg_duration': df['duration'].mean(),
        'avg_waiting_time': df['waitingTime'].mean(),
        'total_waiting_time': df['waitingTime'].sum(),
        'avg_time_loss': df['timeLoss'].mean(),
        'total_time_loss': df['timeLoss'].sum(),
        'avg_route_length': df['routeLength'].mean(),
        'std_waiting_time': df['waitingTime'].std(),
        'max_waiting_time': df['waitingTime'].max(),
        'min_waiting_time': df['waitingTime'].min(),
        'avg_arrival_speed': df['arrivalSpeed'].mean(),
    }
    
    return metrics, df

# Parse all scenarios
results_data = {}
for scenario_name, scenario_path in SCENARIOS.items():
    tripinfo_path = os.path.join(scenario_path, 'results', 'tripinfo.xml')
    result = parse_tripinfo(tripinfo_path)
    if result:
        metrics, df = result
        results_data[scenario_name] = {'metrics': metrics, 'df': df}
        print(f"\n✓ {scenario_name}: {metrics['num_vehicles']} vehicles processed")
    else:
        print(f"\n✗ {scenario_name}: Could not parse tripinfo.xml")

print("\nData loading complete!")

In [ ]:
import os
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set up plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)

# Define scenario paths
BASE_DIR = os.getcwd()
SCENARIOS = {
    'Baseline': os.path.join(BASE_DIR, 'Original network'),
    'Actuated': os.path.join(BASE_DIR, 'Network with actuated control'),
    'Max Pressure': os.path.join(BASE_DIR, 'Network with max pressure control'),
    'RL': os.path.join(BASE_DIR, 'Network with RL control')
}

print("Scenario paths:")
for name, path in SCENARIOS.items():
    print(f"  {name}: {path}")
    print(f"    Results folder exists: {os.path.exists(os.path.join(path, 'results'))}")

# Scenario Comparison Analysis

This notebook compares the performance of all traffic control strategies:
- **Baseline**: Original network with fixed timing
- **Actuated**: Actuated traffic control
- **Max Pressure**: Max Pressure traffic control
- **RL**: Reinforcement Learning traffic control

The comparison includes KPIs such as average waiting time, queue length, travel time, and emissions.

## Summary Statistics

In [ ]:
# Create summary DataFrame
summary_data = []
for scenario_name, result in results_data.items():
    metrics = result['metrics']
    summary_data.append({
        'Scenario': scenario_name,
        'Num Vehicles': metrics['num_vehicles'],
        'Avg Waiting Time (s)': metrics['avg_waiting_time'],
        'Total Waiting Time (s)': metrics['total_waiting_time'],
        'Avg Time Loss (s)': metrics['avg_time_loss'],
        'Avg Duration (s)': metrics['avg_duration'],
        'Avg Arrival Speed (m/s)': metrics['avg_arrival_speed'],
        'Max Waiting Time (s)': metrics['max_waiting_time'],
    })

summary_df = pd.DataFrame(summary_data).set_index('Scenario')
print("\n" + "="*100)
print("SUMMARY STATISTICS - All Scenarios")
print("="*100)
print(summary_df.round(2).to_string())
print("="*100)

## Comparative Visualization - Key Performance Indicators

In [ ]:
# Create comparison plots
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
fig.suptitle('Traffic Control Comparison - Key Performance Indicators', fontsize=16, fontweight='bold')

# 1. Average Waiting Time
ax = axes[0, 0]
scenarios = summary_df.index
waiting_times = summary_df['Avg Waiting Time (s)']
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']
bars = ax.bar(scenarios, waiting_times, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
ax.set_ylabel('Time (seconds)', fontsize=11, fontweight='bold')
ax.set_title('Average Waiting Time', fontsize=12, fontweight='bold')
ax.grid(axis='y', alpha=0.3)
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.1f}s', ha='center', va='bottom', fontweight='bold')

# 2. Total Waiting Time
ax = axes[0, 1]
total_wait = summary_df['Total Waiting Time (s)']
bars = ax.bar(scenarios, total_wait, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
ax.set_ylabel('Time (seconds)', fontsize=11, fontweight='bold')
ax.set_title('Total Waiting Time (Sum)', fontsize=12, fontweight='bold')
ax.grid(axis='y', alpha=0.3)
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.0f}s', ha='center', va='bottom', fontweight='bold', fontsize=9)

# 3. Average Time Loss
ax = axes[0, 2]
time_loss = summary_df['Avg Time Loss (s)']
bars = ax.bar(scenarios, time_loss, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
ax.set_ylabel('Time (seconds)', fontsize=11, fontweight='bold')
ax.set_title('Average Time Loss', fontsize=12, fontweight='bold')
ax.grid(axis='y', alpha=0.3)
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.1f}s', ha='center', va='bottom', fontweight='bold')

# 4. Average Trip Duration
ax = axes[1, 0]
duration = summary_df['Avg Duration (s)']
bars = ax.bar(scenarios, duration, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
ax.set_ylabel('Time (seconds)', fontsize=11, fontweight='bold')
ax.set_title('Average Trip Duration', fontsize=12, fontweight='bold')
ax.grid(axis='y', alpha=0.3)
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.1f}s', ha='center', va='bottom', fontweight='bold')

# 5. Average Arrival Speed
ax = axes[1, 1]
speed = summary_df['Avg Arrival Speed (m/s)']
bars = ax.bar(scenarios, speed, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
ax.set_ylabel('Speed (m/s)', fontsize=11, fontweight='bold')
ax.set_title('Average Arrival Speed', fontsize=12, fontweight='bold')
ax.grid(axis='y', alpha=0.3)
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.2f}', ha='center', va='bottom', fontweight='bold')

# 6. Max Waiting Time
ax = axes[1, 2]
max_wait = summary_df['Max Waiting Time (s)']
bars = ax.bar(scenarios, max_wait, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
ax.set_ylabel('Time (seconds)', fontsize=11, fontweight='bold')
ax.set_title('Maximum Waiting Time', fontsize=12, fontweight='bold')
ax.grid(axis='y', alpha=0.3)
for i, bar in enumerate(bars):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{height:.1f}s', ha='center', va='bottom', fontweight='bold', fontsize=9)

plt.tight_layout()
plt.show()

print("Comparison plots generated successfully!")

## Distribution Analysis - Waiting Time by Vehicle

In [ ]:
# Create distribution comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('Waiting Time Distribution Comparison', fontsize=14, fontweight='bold')

# Box plot
ax = axes[0]
data_to_plot = [results_data[scenario]['df']['waitingTime'].values for scenario in results_data.keys()]
bp = ax.boxplot(data_to_plot, labels=results_data.keys(), patch_artist=True)
for patch, color in zip(bp['boxes'], ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)
ax.set_ylabel('Waiting Time (seconds)', fontsize=11, fontweight='bold')
ax.set_title('Waiting Time Distribution (Box Plot)', fontsize=12, fontweight='bold')
ax.grid(axis='y', alpha=0.3)

# Histogram
ax = axes[1]
for scenario, color in zip(results_data.keys(), ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']):
    data = results_data[scenario]['df']['waitingTime']
    ax.hist(data, bins=30, alpha=0.5, label=scenario, color=color, edgecolor='black')
ax.set_xlabel('Waiting Time (seconds)', fontsize=11, fontweight='bold')
ax.set_ylabel('Number of Vehicles', fontsize=11, fontweight='bold')
ax.set_title('Waiting Time Distribution (Histogram)', fontsize=12, fontweight='bold')
ax.legend(loc='upper right')
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("Distribution plots generated successfully!")

## Detailed Comparison Table with Improvements

In [ ]:
# Calculate improvements vs baseline
baseline_avg_wait = summary_df.loc['Baseline', 'Avg Waiting Time (s)']
baseline_duration = summary_df.loc['Baseline', 'Avg Duration (s)']

improvements = []
for scenario in summary_df.index:
    if scenario != 'Baseline':
        avg_wait = summary_df.loc[scenario, 'Avg Waiting Time (s)']
        duration = summary_df.loc[scenario, 'Avg Duration (s)']
        speed = summary_df.loc[scenario, 'Avg Arrival Speed (m/s)']
        
        wait_improvement = ((baseline_avg_wait - avg_wait) / baseline_avg_wait) * 100
        duration_improvement = ((baseline_duration - duration) / baseline_duration) * 100
        
        improvements.append({
            'Scenario': scenario,
            'Waiting Time Improvement (%)': wait_improvement,
            'Duration Improvement (%)': duration_improvement,
            'Avg Speed (m/s)': speed,
        })

improvement_df = pd.DataFrame(improvements).set_index('Scenario')
print("\n" + "="*80)
print("IMPROVEMENT vs BASELINE")
print("="*80)
print(improvement_df.round(2).to_string())
print("="*80)

# Visualization of improvements
fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(len(improvement_df.index))
width = 0.35

bars1 = ax.bar(x - width/2, improvement_df['Waiting Time Improvement (%)'], 
               width, label='Waiting Time', color='#4ECDC4', alpha=0.8, edgecolor='black', linewidth=1.5)
bars2 = ax.bar(x + width/2, improvement_df['Duration Improvement (%)'], 
               width, label='Trip Duration', color='#45B7D1', alpha=0.8, edgecolor='black', linewidth=1.5)

ax.set_ylabel('Improvement vs Baseline (%)', fontsize=12, fontweight='bold')
ax.set_title('Performance Improvement Over Baseline Scenario', fontsize=13, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(improvement_df.index)
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
ax.legend(fontsize=11)
ax.grid(axis='y', alpha=0.3)

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}%', ha='center', va='bottom' if height > 0 else 'top', fontweight='bold')

plt.tight_layout()
plt.show()

## Scenario Rankings

In [ ]:
# Create rankings
print("\n" + "="*80)
print("SCENARIO RANKINGS")
print("="*80)

# Waiting time ranking (lower is better)
print("\n1. AVERAGE WAITING TIME (Lower is Better)")
print("-" * 80)
for i, (scenario, wait_time) in enumerate(summary_df['Avg Waiting Time (s)'].sort_values().items(), 1):
    print(f"   {i}. {scenario:20s} {wait_time:8.2f} seconds")

# Time loss ranking (lower is better)
print("\n2. AVERAGE TIME LOSS (Lower is Better)")
print("-" * 80)
for i, (scenario, time_loss) in enumerate(summary_df['Avg Time Loss (s)'].sort_values().items(), 1):
    print(f"   {i}. {scenario:20s} {time_loss:8.2f} seconds")

# Trip duration ranking (lower is better)
print("\n3. AVERAGE TRIP DURATION (Lower is Better)")
print("-" * 80)
for i, (scenario, duration) in enumerate(summary_df['Avg Duration (s)'].sort_values().items(), 1):
    print(f"   {i}. {scenario:20s} {duration:8.2f} seconds")

# Speed ranking (higher is better)
print("\n4. AVERAGE ARRIVAL SPEED (Higher is Better)")
print("-" * 80)
for i, (scenario, speed) in enumerate(summary_df['Avg Arrival Speed (m/s)'].sort_values(ascending=False).items(), 1):
    print(f"   {i}. {scenario:20s} {speed:8.2f} m/s")

print("=" * 80)

## Radar Chart - Multi-Dimensional Comparison

In [ ]:
# Prepare normalized metrics for radar chart
from math import pi

# Normalize metrics (0-1 scale, where 1 is best)
radar_data = {}
categories = ['Waiting Time', 'Time Loss', 'Trip Duration', 'Speed']
num_vars = len(categories)

# Calculate normalized scores
for scenario in summary_df.index:
    scores = []
    
    # Waiting time (lower is better) - invert
    wait = summary_df.loc[scenario, 'Avg Waiting Time (s)']
    max_wait = summary_df['Avg Waiting Time (s)'].max()
    scores.append(1 - (wait / max_wait))
    
    # Time loss (lower is better) - invert
    loss = summary_df.loc[scenario, 'Avg Time Loss (s)']
    max_loss = summary_df['Avg Time Loss (s)'].max()
    scores.append(1 - (loss / max_loss))
    
    # Trip duration (lower is better) - invert
    dur = summary_df.loc[scenario, 'Avg Duration (s)']
    max_dur = summary_df['Avg Duration (s)'].max()
    scores.append(1 - (dur / max_dur))
    
    # Speed (higher is better)
    spd = summary_df.loc[scenario, 'Avg Arrival Speed (m/s)']
    max_spd = summary_df['Avg Arrival Speed (m/s)'].max()
    scores.append(spd / max_spd)
    
    radar_data[scenario] = scores

# Create radar chart
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

angles = [n / float(num_vars) * 2 * pi for n in range(num_vars)]
angles += angles[:1]

colors = {'Baseline': '#FF6B6B', 'Actuated': '#4ECDC4', 'Max Pressure': '#45B7D1', 'RL': '#96CEB4'}

for scenario, scores in radar_data.items():
    values = scores + scores[:1]
    ax.plot(angles, values, 'o-', linewidth=2, label=scenario, color=colors.get(scenario, '#999'))
    ax.fill(angles, values, alpha=0.15, color=colors.get(scenario, '#999'))

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories, size=11, fontweight='bold')
ax.set_ylim(0, 1)
ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
ax.set_yticklabels(['0.2', '0.4', '0.6', '0.8', '1.0'], size=9)
ax.grid(True, linestyle='--', alpha=0.7)
plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=11)
plt.title('Multi-Dimensional Performance Comparison\n(Higher values = Better Performance)', 
          fontsize=13, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

print("Radar chart generated successfully!")

## Summary and Key Findings

In [ ]:
print("\n" + "="*100)
print("COMPARISON SUMMARY AND KEY FINDINGS")
print("="*100)

print("\n📊 DATA OVERVIEW")
print("-" * 100)
print(f"Total scenarios analyzed: {len(results_data)}")
print(f"Total vehicles simulated: {summary_df['Num Vehicles'].sum():,}")
print(f"Simulation duration: 3600 seconds (1 hour per scenario)")

print("\n🏆 BEST PERFORMERS")
print("-" * 100)

# Find best scenario for each metric
best_wait = summary_df['Avg Waiting Time (s)'].idxmin()
best_loss = summary_df['Avg Time Loss (s)'].idxmin()
best_duration = summary_df['Avg Duration (s)'].idxmin()
best_speed = summary_df['Avg Arrival Speed (m/s)'].idxmax()

print(f"✓ Lowest Avg Waiting Time:   {best_wait:20s} ({summary_df.loc[best_wait, 'Avg Waiting Time (s)']:.2f}s)")
print(f"✓ Lowest Avg Time Loss:      {best_loss:20s} ({summary_df.loc[best_loss, 'Avg Time Loss (s)']:.2f}s)")
print(f"✓ Lowest Trip Duration:      {best_duration:20s} ({summary_df.loc[best_duration, 'Avg Duration (s)']:.2f}s)")
print(f"✓ Highest Arrival Speed:     {best_speed:20s} ({summary_df.loc[best_speed, 'Avg Arrival Speed (m/s)']:.2f}m/s)")

print("\n📈 KEY STATISTICS")
print("-" * 100)
total_wait = summary_df['Total Waiting Time (s)'].sum()
total_loss = summary_df['Total Time Loss (s)'].sum()
print(f"Total system waiting time:   {total_wait:,.0f} seconds across all scenarios")
print(f"Total system time loss:      {total_loss:,.0f} seconds across all scenarios")

print("\n💡 OBSERVATIONS")
print("-" * 100)
if 'Baseline' in summary_df.index:
    baseline_wait = summary_df.loc['Baseline', 'Avg Waiting Time (s)']
    for scenario in summary_df.index:
        if scenario != 'Baseline':
            wait_time = summary_df.loc[scenario, 'Avg Waiting Time (s)']
            diff = baseline_wait - wait_time
            pct = (diff / baseline_wait) * 100
            trend = "✓ BETTER" if diff > 0 else "✗ WORSE"
            print(f"{scenario:20s}: {trend:10s} (Δ {diff:+6.2f}s, {pct:+6.1f}%)")

print("\n" + "="*100)